In [ ]:
import pandas as pd

# Load the data
df_train = pd.read_csv('/content/drive/MyDrive/_COURSES_/PBA/Preprocessing/train_preprocessed-stationarized.csv')
df_test = pd.read_csv('/content/drive/MyDrive/_COURSES_/PBA/Preprocessing/test_preprocessed-5')

# Display the first few rows of each dataset to understand their structure
df_train_head = df_train.head()
df_test_head = df_test.head()

df_train_head, df_test_head


(   store_nbr  product_type        date  sales  special_offer  day of week  \
 0          1    AUTOMOTIVE  2017-01-01    0.0              0            7   
 1          1        BEAUTY  2017-01-01    0.0              0            7   
 2          1     BEVERAGES  2017-01-01    0.0              0            7   
 3          1         BOOKS  2017-01-01    0.0              0            7   
 4          1  BREAD/BAKERY  2017-01-01    0.0              0            7   
 
    sales_lag7  sales_lag30  sales_lag365  rolling_means7  encoded_product_type  
 0       6.000        3.000           0.0        3.714286                     0  
 1       2.000        6.000           0.0        2.285714                     2  
 2    2576.000     2647.000           0.0     1758.857143                     3  
 3       8.000        1.000           0.0        0.428571                     4  
 4     353.597      371.051           0.0      284.616143                     5  ,
    store_nbr  product_type        da

In [ ]:
# Convert 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

# Create 'year', 'month', and 'day' columns
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day

df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day

# 'sales' is your target variable
y_train = df_train['sales']
X_train = df_train.drop('sales', axis=1)

y_test = df_test['sales']
X_test = df_test.drop('sales', axis=1)


In [ ]:
df_test

,store_nbr,product_type,sales,special_offer,day of week,sales_lag7,sales_lag30,sales_lag365,rolling_means7,encoded_product_type,year,month,day,predicted_sales
date,,,,,,,,,,,,,,
2017-07-31,1,AUTOMOTIVE,8.000,0,1,4.000,7.000,9.000,5.285714,0,2017,7,31,4.932494
2017-07-31,1,BEAUTY,3.000,0,1,1.000,7.000,4.000,3.000000,2,2017,7,31,-2.758429
2017-07-31,1,BEVERAGES,2414.000,24,1,2158.000,2596.000,2286.000,2118.142857,3,2017,7,31,2118.390241
2017-07-31,1,BOOKS,1.000,0,1,0.000,0.000,0.000,0.142857,4,2017,7,31,-1792.879270
2017-07-31,1,BREAD/BAKERY,370.994,0,1,365.350,344.722,442.965,329.796720,5,2017,7,31,14.952111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-15,54,POULTRY,59.619,0,2,73.289,103.902,82.331,66.849714,28,2017,8,15,61.272106
2017-08-15,54,PREPARED FOODS,94.000,0,2,86.000,50.000,90.000,88.428571,29,2017,8,15,93.317826
2017-08-15,54,PRODUCE,915.371,76,2,790.013,981.712,708.864,673.538143,30,2017,8,15,618.765510


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Select the columns
X_train = df_train[['special_offer', 'store_nbr','encoded_product_type','sales_lag365','sales_lag7','rolling_means7']].dropna()
y_train = df_train.loc[X_train.index, 'sales']
X_test = df_test[['special_offer', 'store_nbr','encoded_product_type','sales_lag365','sales_lag7','rolling_means7']].dropna()
y_test = df_test.loc[X_test.index, 'sales']

# Fit SARIMA model on the training data
sarima_model = SARIMAX(df_train['sales'],
                       order=(1, 1, 1),  # Example parameters
                       seasonal_order=(1, 1, 1, 7),  # Example parameters
                       enforce_stationarity=False,
                       enforce_invertibility=False)
sarima_result = sarima_model.fit()

# Predict on training data
train_preds = sarima_result.predict(start=df_train.index[0], end=df_train.index[-1])

# Predict on test data
# Note: SARIMA uses historical data, so predictions on test data might require the train data
test_preds = sarima_result.predict(start=df_test.index[0], end=df_test.index[-1])

# Calculate MAE
train_mae = mean_absolute_error(df_train['sales'], train_preds)
test_mae = mean_absolute_error(df_test['sales'], test_preds)

# Calculate RMSE
train_rmse = mean_squared_error(df_train['sales'], train_preds, squared=False)
test_rmse = mean_squared_error(df_test['sales'], test_preds, squared=False)

# R2 Score is not typically used in time series forecasting, but can be calculated as a reference
train_r2 = r2_score(df_train['sales'], train_preds)
test_r2 = r2_score(df_test['sales'], test_preds)

print(f'MAE on train set: {train_mae}')
print(f'MAE on test set: {test_mae}\n')
print(f'RMSE on train set: {train_rmse}')
print(f'RMSE on test set: {test_rmse}\n')
print(f'R2 on train set: {train_r2}')
print(f'R2 on test set: {test_r2}\n')


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MAE on train set: 689.2974189928684
MAE on test set: 696.502339543237

RMSE on train set: 1380.6621651190562
RMSE on test set: 1299.924098980283

R2 on train set: 0.000532840759801978
R2 on test set: -0.05658869555932



In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assuming 'sales' is your target variable and the rest are exogenous variables
# Fit a SARIMAX model
sarimax_model = SARIMAX(df_train['sales'],
                        exog=df_train[['special_offer', 'store_nbr','encoded_product_type','sales_lag365','sales_lag7','rolling_means7']],
                        order=(1, 1, 1),
                        seasonal_order=(1, 1, 1, 7),
                        enforce_stationarity=False,
                        enforce_invertibility=False).fit()

# Make predictions for the test set
test_exog = df_test[['special_offer', 'store_nbr','encoded_product_type','sales_lag365','sales_lag7','rolling_means7']]
test_preds = sarimax_model.predict(start=df_test.index[0], end=df_test.index[-1], exog=test_exog)

# Add the predictions to the test DataFrame
df_test['predicted_sales'] = test_preds

# Display the date, predicted value of each product at each store
print(df_test[['date', 'store_nbr', 'encoded_product_type','sales', 'predicted_sales']])


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


            date  store_nbr  encoded_product_type     sales  predicted_sales
0     2017-07-31          1                     0     8.000         4.932494
1     2017-07-31          1                     2     3.000        -2.758429
2     2017-07-31          1                     3  2414.000      2118.390241
3     2017-07-31          1                     4     1.000     -1792.879270
4     2017-07-31          1                     5   370.994        14.952111
...          ...        ...                   ...       ...              ...
27659 2017-08-15         54                    28    59.619        61.272106
27660 2017-08-15         54                    29    94.000        93.317826
27661 2017-08-15         54                    30   915.371       618.765510
27662 2017-08-15         54                    31     0.000         7.951129
27663 2017-08-15         54                    32     3.000        20.458317

[27664 rows x 5 columns]


In [ ]:
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# Add predictions to the DataFrame
df_train['predicted_sales'] = train_preds

df_train.set_index('date', inplace=True)

# Since df_train's index is already set to 'date', you can directly resample
train_result_agg = df_train.resample('D')[['sales', 'predicted_sales']].mean().reset_index()

# Plotting using Plotly
import plotly.express as px

fig = px.line(train_result_agg, x='date', y='sales', title='Actual vs Prediction Sales on Train dataset')
fig.add_scatter(x=train_result_agg['date'], y=train_result_agg['predicted_sales'], mode='lines', name='Predicted Sales')
fig.show()


In [ ]:
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# Add predictions to the DataFrame
df_test['predicted_sales'] = test_preds

df_test.set_index('date', inplace=True)

# Since df_train's index is already set to 'date', you can directly resample
test_result_agg = df_test.resample('D')[['sales', 'predicted_sales']].mean().reset_index()

# Plotting using Plotly
import plotly.express as px

fig = px.line(test_result_agg, x='date', y='sales', title='Actual vs Prediction Sales on Test dataset')
fig.add_scatter(x=test_result_agg['date'], y=test_result_agg['predicted_sales'], mode='lines', name='Predicted Sales')
fig.show()


In [ ]:
store_train = df_test.groupby(['store_nbr', 'date'])[['sales', 'predicted_sales']].mean().reset_index()
store_df = store_train
fig = make_subplots(rows=18, cols=3, subplot_titles=[f'Store {store}' for store in store_train.store_nbr.unique()])
n=1
for row in range (1,19):
  for col in range(1,4):
    df = store_df[store_df['store_nbr'] == n]
    n += 1

    px_fig = px.line(df, x='date', y='sales')
    px_fig.add_scatter(x=df['date'], y=df['predicted_sales'], mode='lines')

    for trace in px_fig['data']:
      fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=2000, title_text = 'Sales Prediction by Stores on Test dataset')

fig.show()


In [ ]:
product_test = df_test.groupby(['encoded_product_type', 'product_type', 'date'])[['sales', 'predicted_sales']].mean().reset_index()
product_df = product_test
fig = make_subplots(rows=11, cols=3, subplot_titles=[f'{product}' for product in product_test.product_type.unique()])
n=0
for row in range (1,12):
  for col in range(1,4):
    df = product_df[product_df['encoded_product_type'] == n]
    n += 1

    px_fig = px.line(df, x='date', y='sales')
    px_fig.add_scatter(x=df['date'], y=df['predicted_sales'], mode='lines')

    for trace in px_fig['data']:
      fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=2000, title_text = 'Sales Prediction by Product on Test dataset')

fig.show()
